## Import Classes from Libraries

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

## Read Tangled Prompt and Spec from org-mode export

In [2]:
format_instructions = open("formatter-prompt.txt", "r")
description = open("description.txt", "r")

In [3]:
model = ChatOpenAI(model="gpt-4-turbo-preview")
spec_prompt =  ChatPromptTemplate.from_messages([
  ("system", format_instructions.read()),
  ("user", "Convert the following data flow description to an interaction list: {description}")
])
parser = StrOutputParser()

## Invoke the LLM Chat Model

In [4]:
spec_chain = spec_prompt | model | parser

In [5]:
spec =  spec_chain.invoke({"description": description.read()})

In [6]:
print(spec)

- user sends diagram description (plain text) to orchestrator service
- orchestrator service sends prompt (plain text) to formatter LLM
- formatter LLM sends bulleted list of interactions to orchestrator service
- orchestrator service sends interaction list to diagrammer LLM
- diagrammer LLM sends diagram source code to orchestrator service
- orchestrator service sends diagram source code to diagramming tool
- diagramming tool sends diagram image (PNG) to orchestrator service
- orchestrator service sends diagram image (PNG) to user


## Diagram Generation


In [7]:
diagram_instructions = open("diagrammer-prompt.txt", "r")
diagram_prompt = ChatPromptTemplate.from_messages([
  ("system", diagram_instructions.read()),
  ("user", "Diagram the following data flow: {spec}")
])

In [8]:
diagram_chain = diagram_prompt | model | parser

In [9]:
diagram_source = diagram_chain.invoke({"spec": spec})

In [10]:
print(diagram_source)

```dot
digraph G {
    graph [bgcolor=white]
    node [fontname="Roboto Mono", colorscheme=paired12, style=filled];
    
    user [shape=oval, color=1];
    "orchestrator service" [shape=box, color=2];
    "formatter LLM" [shape=box, color=3];
    "diagrammer LLM" [shape=box, color=4];
    "diagramming tool" [shape=box, color=5];

    user -> "orchestrator service" [label="diagram description (plain text)"];
    "orchestrator service" -> "formatter LLM" [label="prompt (plain text)"];
    "formatter LLM" -> "orchestrator service" [label="bulleted list of interactions"];
    "orchestrator service" -> "diagrammer LLM" [label="interaction list"];
    "diagrammer LLM" -> "orchestrator service" [label="diagram source code"];
    "orchestrator service" -> "diagramming tool" [label="diagram source code"];
    "diagramming tool" -> "orchestrator service" [label="diagram image (PNG)"];
    "orchestrator service" -> user [label="diagram image (PNG)"];
}
```
